# Data Validation

In [19]:
import os
from dataclasses import dataclass
from pathlib import Path
from mlProject.constants import *  # fetching paths to config, shcema, parans yaml files etc
from mlProject.utils.common import read_yaml, create_directories
from mlProject import logger

In [2]:
pwd

'd:\\projects\\MLflow_DL\\Machine-learning-project-with-MLflow\\research'

In [3]:
os.chdir("..")

In [4]:
pwd

'd:\\projects\\MLflow_DL\\Machine-learning-project-with-MLflow'

In [56]:
import pandas as pd


df = pd.read_csv(r"artifacts\data_ingestion\winequality-red.csv", sep = ';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [57]:
df.shape

(1599, 12)

In [58]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

lets create the entity first

In [59]:


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    unzip_data_dir:Path
    STATUS_FILE:str
    all_schema: dict

now we define our configurations manager for this part of the pipeline just as we did in dataa ingestion

In [60]:



class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.schema = read_yaml(schema_filepath)
            self.params = read_yaml(params_filepath)
        #     print(self.schema)

            create_directories([self.config.artifacts_root])
    

    def get_data_validation_config(self) -> DataValidationConfig:
          config = self.config.data_validation
          schema = self.schema.COLUMNS

          
          create_directories([config.root_dir])

          data_validation_config = DataValidationConfig(
                  root_dir = config.root_dir,
                  unzip_data_dir = config.unzip_data_dir,
                  STATUS_FILE = config.STATUS_FILE,
                  all_schema = schema,
      
          )
          return data_validation_config


In [63]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir, sep=";")
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                
            return validation_status
        
        except Exception as e:
            raise e
        
        
    def validate_all_datatypes(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir, sep=";")
            data_column_types = {col: str(data[col].dtype) for col in data.columns}
            all_schema = self.config.all_schema

            for column in all_schema:
                if (column not in data_column_types) or (data_column_types[column]!=all_schema[column]):
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    return validation_status
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                
            return validation_status
        
        except Exception as e:
            raise e
        

In [64]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_all_datatypes()

    
except Exception as e:
    raise e

[2025-01-23 11:46:50,110: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-23 11:46:50,113: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-23 11:46:50,121: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-23 11:46:50,121: INFO: common: created directory at: artifacts]
[2025-01-23 11:46:50,129: INFO: common: created directory at: artifacts/data_validation]
dataset info {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}
schema info {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'floa

In [65]:
x =  {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}
y =  {'volatile acidity': 'float64','fixed acidity': 'floa64',  'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}

validation=False
def check():
    for key in x:
        print(f"checking {key}")
        if (key not in y) or y[key]!=x[key]:
            validation = False
            print(f"{key} not in y and {y[key]} != {x[key]}")
            return validation
        else:
            validation = True
            print(f"value match")
    return validation
        

print(f"Validation: {validation}")


Validation: False
